In [10]:
# !pip install --upgrade pip
# !pip install matplotlib
# !pip install -U scikit-learn
# !pip install sktime
# !pip install torch torchvision torchaudio
# !pip install transformers
# !pip install pytesseract
# !pip install opencv-python
# !pip install seaborn

In [11]:
from core import data_manager, settings
from core.utils import setup_logging
from pathlib import Path

settings.logging.level = "DEBUG"

setup_logging()

In [12]:
# data_manager.backup_data(paths=[Path("/Users/igor/Desktop/App-parser/data/raw/")])

In [13]:
coins = {}

for coin in data_manager.coin_list:
    # if coin != "TON":
    #     continue
    for path in data_manager.get_path(data_type="raw",
                                      coin=coin,
                                      timetravel="5m"):
        coins.setdefault(coin, [])
        coins[coin].append(path)

In [14]:
from backend.Dataset import DatasetTimeseries
import pandas as pd

In [15]:
coins_clear = {}

for coin in data_manager.coin_list:
    # if coin != "TON":
    #     continue
    for path in data_manager.get_path(data_type="processed",
                                      coin=coin,
                                      timetravel="5m"):
        coins_clear.setdefault(coin, [])
        coins_clear[coin].append(path)

In [16]:
coins_clear

{'SOL': [PosixPath('/Users/igor/Desktop/App-parser/data/processed/SOL/5m/clear_SOL_5m.csv')],
 'XRP': [PosixPath('/Users/igor/Desktop/App-parser/data/processed/XRP/5m/clear_XRP_5m.csv')],
 'BNB': [PosixPath('/Users/igor/Desktop/App-parser/data/processed/BNB/5m/clear_BNB_5m.csv')],
 'BTC': [PosixPath('/Users/igor/Desktop/App-parser/data/processed/BTC/5m/clear_BTC_5m.csv')],
 'XMR': [PosixPath('/Users/igor/Desktop/App-parser/data/processed/XMR/5m/clear_XMR_5m.csv')],
 'ETH': [PosixPath('/Users/igor/Desktop/App-parser/data/processed/ETH/5m/clear_ETH_5m.csv')]}

In [17]:
coins

{'SHIB': [PosixPath('/Users/igor/Desktop/App-parser/data/raw/launch_parser_10/SHIB/SHIB_5m.csv'),
  PosixPath('/Users/igor/Desktop/App-parser/data/raw/launch_parser_11/SHIB/SHIB_5m.csv'),
  PosixPath('/Users/igor/Desktop/App-parser/data/raw/launch_parser_1/SHIB/SHIB_5m.csv'),
  PosixPath('/Users/igor/Desktop/App-parser/data/raw/launch_parser_6/SHIB/SHIB_5m.csv'),
  PosixPath('/Users/igor/Desktop/App-parser/data/raw/launch_parser_7/SHIB/SHIB_5m.csv'),
  PosixPath('/Users/igor/Desktop/App-parser/data/raw/launch_parser_13/SHIB/SHIB_5m.csv'),
  PosixPath('/Users/igor/Desktop/App-parser/data/raw/launch_parser_14/SHIB/SHIB_5m.csv'),
  PosixPath('/Users/igor/Desktop/App-parser/data/raw/launch_parser_15/SHIB/SHIB_5m.csv'),
  PosixPath('/Users/igor/Desktop/App-parser/data/raw/launch_parser_12/SHIB/SHIB_5m.csv'),
  PosixPath('/Users/igor/Desktop/App-parser/data/raw/launch_parser_5/SHIB/SHIB_5m.csv'),
  PosixPath('/Users/igor/Desktop/App-parser/data/raw/launch_parser_2/SHIB/SHIB_5m.csv'),
  Posix

In [19]:
coins_dt = {}
coins_dubl = {}
for coin, paths in coins.items():
    for path in paths:
        dt = DatasetTimeseries(path)
        dt.set_dataset(dt.clear_dataset())
        coins_dt.setdefault(coin, [])
        coins_dt[coin].append(dt)

    # df_new = pd.concat(map(lambda dt: dt.get_dataset(), coins_dt[coin]), ignore_index=True)
    # df_new = DatasetTimeseries(df_new).dataset_clear()
    # df_new.drop_duplicates(subset=["datetime", "open", "min", "max", "close", "volume"], inplace=True)
    # df_new = DatasetTimeseries(df_new)
    # coins_dubl.setdefault(coin, len(df_new.duplicated()))

    # df_new.set_dataset(df_new.clear_dataset())
    # path_coin = data_manager.create_dir("processed", coin)
    # df_new.set_path_save(data_manager.create_dir("processed", path_coin / "5m"))
    # df_new.save_dataset(f"clear-{coin}-5m.csv")
    # coins_dt[coin] = df_new
    

Function clear_dataset executed in 0:00:02.624921
Function clear_dataset executed in 0:00:02.624921
Function clear_dataset executed in 0:00:00.273228
Function clear_dataset executed in 0:00:00.273228
Function clear_dataset executed in 0:00:00.007094
Function clear_dataset executed in 0:00:00.007094
Function clear_dataset executed in 0:00:00.010777
Function clear_dataset executed in 0:00:00.010777
Function clear_dataset executed in 0:00:00.009776
Function clear_dataset executed in 0:00:00.009776
Function clear_dataset executed in 0:00:00.007852
Function clear_dataset executed in 0:00:00.007852
Function clear_dataset executed in 0:00:03.525200
Function clear_dataset executed in 0:00:03.525200
Function clear_dataset executed in 0:00:00.010232
Function clear_dataset executed in 0:00:00.010232
Function clear_dataset executed in 0:00:00.053468
Function clear_dataset executed in 0:00:00.053468
Function clear_dataset executed in 0:00:00.008702
Function clear_dataset executed in 0:00:00.008702


In [20]:
def sear_datetime(dt: pd.DataFrame, date):
    return dt[dt["datetime"] == date]


In [24]:
coin_new_dt = {}

for coin, dts in coins_dt.items():
    if coin in ["SHIB"]:
        continue
    min_data = None
    max_data = None
    print("-"* 10)
    print(coin)
    for dt in dts:
        if min_data is None or min_data > dt.get_dataset().datetime.min():
            min_data = dt.get_dataset().datetime.min()
        if max_data is None or max_data < dt.get_dataset().datetime.max():
            max_data = dt.get_dataset().datetime.max()
    if (min_data - max_data).total_seconds() == 0:
        continue
    print(f"min: {min_data}")
    print(f"max: {max_data}")
    print((min_data - max_data).total_seconds())

    new_dt = []
    correct_dt = {}
    
    for date in pd.date_range(min_data, max_data, freq="5min"):
        data_buffer = {}
        for dt in dts:
            result = sear_datetime(dt.get_dataset(), date)
            if len(result) == 0 or result["close"].item() == "x":
                continue
            data_buffer.setdefault(dt, result)

        if len(data_buffer.values()) > 1:
            datas = list(data_buffer.values())
            if len(datas) == 2:
                if datas[0]["open"].item() != "x" and datas[1]["open"].item() == "x":
                    new_dt.append(datas[0])
                elif datas[0]["open"].item() == "x" and datas[1]["open"].item() != "x":
                    new_dt.append(datas[1])
                continue
            
            result = max(data_buffer.items(), key=lambda data: data[1]["volume"].item())[1]
            if result["volume"].item() > 1:
                new_dt.append(result)
                continue

            for data in datas[1:]:
                if data["volume"].item() == datas[0]["volume"].item():
                    continue
                i = 0
                dt_i = {}
                i_dt = {}
                flag_correct = False
                dt_correct = None
                for dt, result in data_buffer.items():
                    print(f"{i}: {result}")
                    dt_i[i] = dt
                    i_dt[dt] = i
                    i += 1

                for dt, result in data_buffer.items():
                    if correct_dt.get(dt, 0) > 50:
                        print("-"* 10)
                        print(f"Coorect - {i_dt[dt]}{dt}: {correct_dt[dt]}")
                        print("-"* 10)
                        flag_correct = True
                        dt_correct = dt
                        break

                s = input()

                if s == "c":
                    break
                elif flag_correct and s == "y" and dt_correct is not None:
                    correct_dt.setdefault(dt, 0)
                    correct_dt[dt] += 1
                    new_dt.append(data_buffer[dt_correct])
                    break
                elif s.isdigit():
                    dt = dt_i[int(s)]
                    correct_dt.setdefault(dt, 0)
                    correct_dt[dt] += 1
                    new_dt.append(datas[int(s)])
                    break
                print("-"* 10)

        else:
            if len(data_buffer.values()) == 0:
                new_dt.append([date, "x", "x", "x", "x", "x"])
            else:
                new_dt.append(list(data_buffer.values())[0])

    coin_new_dt[coin] = new_dt


----------
TRX
min: 2025-02-05 03:40:00
max: 2025-04-23 14:45:00
-6692700.0
----------
AVAX
min: 2024-09-12 15:50:00
max: 2025-04-25 05:05:00
-19401300.0
0:                 datetime    open     max     min   close volume
4520 2025-04-06 16:25:00  17.208  17.224  17.165  17.165   0.01
1:                 datetime    open     max     min   close volume
4923 2025-04-06 16:25:00  17.209  17.253  17.206  17.206    0.0
2:                 datetime    open     max     min   close volume
5336 2025-04-06 16:25:00  17.209  17.253  17.206  17.206    0.0
----------
TON
min: 2024-05-13 07:30:00
max: 2025-04-26 06:30:00
-30063600.0
----------
SOL
min: 2024-05-13 23:35:00
max: 2025-04-22 23:25:00
-29721000.0
----------
ADA
min: 2024-05-14 19:25:00
max: 2025-04-28 03:00:00
-30094500.0
----------
LINK
min: 2025-04-19 01:40:00
max: 2025-04-20 09:00:00
-112800.0
----------
XRP
min: 2025-04-19 01:50:00
max: 2025-04-20 09:10:00
-112800.0
----------
BCH
min: 2025-04-19 01:35:00
max: 2025-04-20 09:00:00
-11310

In [39]:

for coin, raw_dt in coin_new_dt.items():
    new_pd = []
    for data in raw_dt:
        if isinstance(data, list):
            data = [{"datetime": data[0], "open": data[1], "min": data[2], "max": data[3], "close": data[4], "volume": data[5]}]
            new_pd.append(pd.DataFrame(data))
        else:
            new_pd.append(data)
    coin_new_dt[coin] = pd.concat(new_pd, ignore_index=True)
    # coin_new_dt[coin] = pd.DataFrame(raw_dt, columns=["datetime", "open", "min", "max", "close", "volume"])

In [57]:
for coin, new_dt in coin_new_dt.items():
    # new_dt = DatasetTimeseries(dt)
    new_dt.set_dataset(new_dt.clear_dataset())
    coin_new_dt[coin] = new_dt
    path_coin = data_manager.create_dir("processed", coin)
    coin_new_dt[coin].set_path_save(data_manager.create_dir("processed", path_coin / "5m"))
    coin_new_dt[coin].save_dataset(f"clear_{coin}_5m.csv")


Function clear_dataset executed in 0:00:00.409738
Function clear_dataset executed in 0:00:00.409738
Directory created: /Users/igor/Desktop/App-parser/data/processed/TRX
Directory created: /Users/igor/Desktop/App-parser/data/processed/TRX
Directory created: /Users/igor/Desktop/App-parser/data/processed/TRX/5m
Directory created: /Users/igor/Desktop/App-parser/data/processed/TRX/5m
Dataset saved to /Users/igor/Desktop/App-parser/data/processed/TRX/5m/clear_TRX_5m.csv
Dataset saved to /Users/igor/Desktop/App-parser/data/processed/TRX/5m/clear_TRX_5m.csv
Function clear_dataset executed in 0:00:00.918937
Function clear_dataset executed in 0:00:00.918937
Directory created: /Users/igor/Desktop/App-parser/data/processed/AVAX
Directory created: /Users/igor/Desktop/App-parser/data/processed/AVAX
Directory created: /Users/igor/Desktop/App-parser/data/processed/AVAX/5m
Directory created: /Users/igor/Desktop/App-parser/data/processed/AVAX/5m
Dataset saved to /Users/igor/Desktop/App-parser/data/proce

In [59]:
for coin, dt_raw in coin_new_dt.items():
    temp_d = []
    dt_raw = dt_raw.get_dataset()
    dt_raw = dt_raw.sort_values(by=["datetime"])
    for data in dt_raw.iterrows():
        temp_d.append(data[1]["datetime"])

        if len(temp_d) > 1:
            if (temp_d[-1] - temp_d[-2]).total_seconds() != 300:
                print(coin)
                print(temp_d[-1])
                print(temp_d[-2])
                print((temp_d[-1] - temp_d[-2]).total_seconds())
                print("-"* 10)
            temp_d.pop(0)


In [30]:
for coin, clear_dt in coins_clear.items():
    coins_clear[coin] = DatasetTimeseries(clear_dt[0])

clear_SOL_5m.csv
clear_XRP_5m.csv
clear_BNB_5m.csv
clear_BTC_5m.csv
clear_XMR_5m.csv
clear_ETH_5m.csv


In [ ]:
for coin, new_dt in coin_new_dt.items():
    new_dt = pd.concat(new_dt, ignore_index=True)
    new_dt = DatasetTimeseries(new_dt).dataset_clear()
    new_dt.drop_duplicates(subset=["datetime", "open", "min", "max", "close", "volume"], inplace=True)
    new_dt = DatasetTimeseries(new_dt)
    coins_dubl.setdefault(coin, len(new_dt.duplicated()))

    new_dt.set_dataset(new_dt.clear_dataset())
    path_coin = data_manager.create_dir("processed", coin)
    new_dt.set_path_save(data_manager.create_dir("processed", path_coin / "5m"))
    new_dt.save_dataset(f"clear_{coin}_5m.csv")
    coin_new_dt[coin] = new_dt

Function clear_dataset executed in 0:00:01.058074
Function clear_dataset executed in 0:00:01.058074
Function clear_dataset executed in 0:00:01.058074
Directory created: /Users/igor/Desktop/App-parser/data/processed/SOL
Directory created: /Users/igor/Desktop/App-parser/data/processed/SOL
Directory created: /Users/igor/Desktop/App-parser/data/processed/SOL
Directory created: /Users/igor/Desktop/App-parser/data/processed/SOL/5m
Directory created: /Users/igor/Desktop/App-parser/data/processed/SOL/5m
Directory created: /Users/igor/Desktop/App-parser/data/processed/SOL/5m
Dataset saved to /Users/igor/Desktop/App-parser/data/processed/SOL/5m/clear_SOL_5m.csv
Dataset saved to /Users/igor/Desktop/App-parser/data/processed/SOL/5m/clear_SOL_5m.csv
Dataset saved to /Users/igor/Desktop/App-parser/data/processed/SOL/5m/clear_SOL_5m.csv
Function clear_dataset executed in 0:00:02.095894
Function clear_dataset executed in 0:00:02.095894
Function clear_dataset executed in 0:00:02.095894
Directory create

In [ ]:
for coin, count in coins_dubl.items():
    if count == 0:
        continue
    print(f"Coin: {coin}, Duplicates: {count}")

In [ ]:
for coin, dt in coins_dt.items():
    if (len(dt.get_dataset_Nan()) / len(dt.get_dataset())) < 0.3:
        print(f"{coin=}\n{dt.get_dataset().shape=}\n{dt.get_dataset_Nan().shape=}")
        print(len(dt.get_dataset_Nan()) / len(dt.get_dataset()))
        print()
    

coin='SHIB'
dt.get_dataset().shape=(133385, 6)
dt.get_dataset_Nan().shape=(45, 6)
0.00033736926940810435

coin='TRX'
dt.get_dataset().shape=(22310, 6)
dt.get_dataset_Nan().shape=(24, 6)
0.0010757507844016136

coin='AVAX'
dt.get_dataset().shape=(64672, 6)
dt.get_dataset_Nan().shape=(41, 6)
0.0006339683325086591

coin='TON'
dt.get_dataset().shape=(100213, 6)
dt.get_dataset_Nan().shape=(749, 6)
0.007474080209154501

coin='SOL'
dt.get_dataset().shape=(99071, 6)
dt.get_dataset_Nan().shape=(176, 6)
0.0017765037195546628

coin='ADA'
dt.get_dataset().shape=(100316, 6)
dt.get_dataset_Nan().shape=(1624, 6)
0.01618884325531321

coin='ETH'
dt.get_dataset().shape=(6474, 6)
dt.get_dataset_Nan().shape=(85, 6)
0.013129440840284215



In [ ]:
coins_clear = {}

for coin in data_manager.coin_list:
    for path in data_manager.get_path(data_type="processed",
                                      coin=coin,
                                      timetravel="5m"):
        coins_clear[coin] = DatasetTimeseries(path)

In [ ]:
coins_clear

{'SHIB': <backend.Dataset.dataset.DatasetTimeseries at 0x1586fc8f0>,
 'TRX': <backend.Dataset.dataset.DatasetTimeseries at 0x1586feab0>,
 'AVAX': <backend.Dataset.dataset.DatasetTimeseries at 0x158c97bc0>,
 'TON': <backend.Dataset.dataset.DatasetTimeseries at 0x158782cf0>,
 'SOL': <backend.Dataset.dataset.DatasetTimeseries at 0x1586ff230>,
 'ADA': <backend.Dataset.dataset.DatasetTimeseries at 0x15828d460>,
 'LINK': <backend.Dataset.dataset.DatasetTimeseries at 0x174c63c50>,
 'XRP': <backend.Dataset.dataset.DatasetTimeseries at 0x17582d1f0>,
 'BCH': <backend.Dataset.dataset.DatasetTimeseries at 0x174c63ce0>,
 'BNB': <backend.Dataset.dataset.DatasetTimeseries at 0x158782b10>,
 'DOT': <backend.Dataset.dataset.DatasetTimeseries at 0x11a5c2a20>,
 'APT': <backend.Dataset.dataset.DatasetTimeseries at 0x1588e4290>,
 'SUI': <backend.Dataset.dataset.DatasetTimeseries at 0x169848860>,
 'LTC': <backend.Dataset.dataset.DatasetTimeseries at 0x173389550>,
 'NEAR': <backend.Dataset.dataset.DatasetTime

In [ ]:
count = 0
count_dubl = 0
#447 1761
for coin, dt in coins_dt.items():
    
    df = dt.get_dataset()
    # df['datetime'] = df['datetime'] - pd.Timedelta(hours=3)
    dt.set_dataset(df)  
   
    dt_clear = coins_clear[coin]
    # print(coin)
    for data in dt_clear:
        if data["close"] == "x":
            continue
            if data["datetime"] in dt.get_dataset()["datetime"].values:
                result = dt.get_dataset()[dt.get_dataset()['datetime'] ==  data["datetime"]]
                if "x" in result["close"].values:
                    continue
                count += 1
        else:
            if data["datetime"] in dt.get_dataset()["datetime"].values:
                result = dt.get_dataset()[dt.get_dataset()['datetime'] ==  data["datetime"]]
                if data["open"] in result["open"].values:
                    print(data)
                    print(result)
                    input()
                    # if data["min"] in result["min"].values:
                    #     if data["max"] in result["max"].values:
                    if data["close"] in result["close"].values:
                                count_dubl += 1


print(count)
print(count_dubl)


KeyboardInterrupt: 

In [ ]:
from backend.MMM import Agent, AgentPReadTime
from backend.Dataset import Indicators
from backend.train_models import Loader

In [ ]:
def transform_data(bath: pd.DataFrame) -> pd.DataFrame:
    bath["SMA20"] = Indicators.sma(bath, 20)
    bath['EMA20'] = Indicators.ema(bath, 20)
    bath['RSI14'] = Indicators.rsi(bath)
    bath['MACD'], bath['Signal'] = Indicators.macd(bath)
    bath['UpperBB'], bath['MiddleBB'], bath['LowerBB'] = Indicators.bollinger_bands(bath)
    bath['ATR14'] = Indicators.atr(bath)
    bath['%K'], bath['%D'] = Indicators.stochastic_oscillator(bath)
   
    return bath
    

In [ ]:
coins_dt

{'TON': <backend.Dataset.dataset.DatasetTimeseries at 0x15a8e70b0>}

In [ ]:
coin_loader = {}
for coin, dt in coins_dt.items():
    dt: DatasetTimeseries
    filter_func = lambda x: x["open"] != "x"

    loader = dt.get_time_line_loader(time_line_size=100, 
                                filter_data=filter_func)
    
    coin_loader[coin] = loader

loader_train = Loader("agent_pred_train")
agent_manager = loader_train.load_model(count_agents=1000)


/Users/igor/Library/Caches/pypoetry/virtualenvs/postgresdocker-MncL8gKv-py3.12/lib/python3.12/site-packages/torch/amp/grad_scaler.py:136: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
Loading Agent: agent_pred_train
Loading schema for: agent_pred_train
Loading multi agent: agent_pred_train


In [ ]:
# agent_manager.agent[0].model

In [ ]:
agent = agent_manager.agent[0]
batch_size = 100
mixed = True
loader = loader_train.load_agent_data(list(coin_loader.values()), agent, batch_size, mixed)

In [ ]:
for x, y, time_x in loader:
    print(x.shape, y.shape, time_x.shape)
    print(y[0][0])
    print(y[0][0])
    break

torch.Size([100, 30, 20]) torch.Size([100, 5, 20]) torch.Size([100, 30, 5])
tensor([ 3.1247e+00,  3.1260e+00,  3.1241e+00,  3.1249e+00,  1.0000e-02,
                nan,  3.1240e+00,  5.2401e+01,  8.3333e-03, -7.5096e-03,
        -8.5506e-03,         nan,         nan,  3.2754e+00,  3.1256e+00,
         2.9759e+00,         nan,  6.0000e-02,  5.0000e+01,  1.5568e+00])
tensor([ 3.1247e+00,  3.1260e+00,  3.1241e+00,  3.1249e+00,  1.0000e-02,
                nan,  3.1240e+00,  5.2401e+01,  8.3333e-03, -7.5096e-03,
        -8.5506e-03,         nan,         nan,  3.2754e+00,  3.1256e+00,
         2.9759e+00,         nan,  6.0000e-02,  5.0000e+01,  1.5568e+00])


In [ ]:
loader_train.train_model(list(coin_loader.values()),
                                  agent_manager=agent_manager,
                                  mixed=True)

🚀 Starting ensemble training
🏋️ Training Agent 1:   0%|          | 0/6 [00:00<?, ?it/s]

⚠️ Mixed precision на MPS может работать некорректно. Рекомендуется использовать fp32


/Users/igor/Library/Caches/pypoetry/virtualenvs/postgresdocker-MncL8gKv-py3.12/lib/python3.12/site-packages/torch/amp/grad_scaler.py:136: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
🏋️ Training Agent 1:   0%|          | 0/6 [00:07<?, ?it/s]


TypeError: Agent.trade() takes 2 positional arguments but 3 were given

In [ ]:
    # @staticmethod
import numpy as np
def calculate_realized_volatility(prices, window=20, trading_days=252):
    """
    prices: Series с ценами закрытия
    window: период расчета волатильности (дни)
    trading_days: число торговых дней в году (для annualization)
    """
    log_returns = np.log(prices / prices.shift(1))
    daily_vol = log_returns.rolling(window).std()
    annualized_vol = daily_vol * np.sqrt(trading_days)
    return annualized_vol

In [ ]:
dt: DatasetTimeseries = coins_dt["TON"]

filter_func = lambda x: x["open"] != "x"
loader = dt.get_time_line_loader(time_line_size=100, 
                                filter_data=filter_func)


In [ ]:
# print(len(loader))
buffer = []
for window in loader:
    # print(window)
    # dt.plot_series(window, param="close")
    crv = calculate_realized_volatility(window["close"],
                                        window=50,
                                        trading_days=60*24*252)
    buffer.append(crv)
    
    # print(crv)
    # break


               datetime    open     max     min   close  volume
508 2025-04-24 12:10:00  3.1012  3.1039  3.0991  3.1038    0.01
509 2025-04-24 12:05:00  3.1070  3.1081  3.0999  3.1013    0.02
510 2025-04-24 12:00:00  3.1115  3.1115  3.1058  3.1066    0.00
511 2025-04-24 11:55:00  3.1120  3.1143  3.1070  3.1106    0.01
512 2025-04-24 11:50:00  3.1071  3.1122  3.1071  3.1091    0.01
..                  ...     ...     ...     ...     ...     ...
603 2025-04-24 04:15:00  3.1138  3.1197  3.1082  3.1141    0.03
604 2025-04-24 04:10:00  3.1099  3.1147  3.0970  3.1137    0.06
605 2025-04-24 04:05:00  3.1198  3.1199  3.1075  3.1085    0.02
606 2025-04-24 04:00:00  3.1028  3.1195  3.1028  3.1184    0.03
607 2025-04-24 03:55:00  3.1110  3.1143  3.1000  3.1034    0.03

[100 rows x 6 columns]
               datetime    open     max     min   close  volume
639 2025-04-24 01:15:00  3.1482  3.1528  3.1433  3.1483    0.01
640 2025-04-24 01:10:00  3.1588  3.1594  3.1461  3.1469    0.01
641 2025-04-24 0

In [ ]:
for crv, window in zip(buffer, loader):
    print(crv)
    print(window)
    dt.plot_series(window, param="close",
                   indicators={"CRV": crv, "on": False})
    

508         NaN
509         NaN
510         NaN
511         NaN
512         NaN
         ...   
603    1.469404
604    1.453181
605    1.461029
606    1.469392
607    1.519271
Name: close, Length: 100, dtype: float64
               datetime    open     max     min   close  volume
508 2025-04-24 12:10:00  3.1012  3.1039  3.0991  3.1038    0.01
509 2025-04-24 12:05:00  3.1070  3.1081  3.0999  3.1013    0.02
510 2025-04-24 12:00:00  3.1115  3.1115  3.1058  3.1066    0.00
511 2025-04-24 11:55:00  3.1120  3.1143  3.1070  3.1106    0.01
512 2025-04-24 11:50:00  3.1071  3.1122  3.1071  3.1091    0.01
..                  ...     ...     ...     ...     ...     ...
603 2025-04-24 04:15:00  3.1138  3.1197  3.1082  3.1141    0.03
604 2025-04-24 04:10:00  3.1099  3.1147  3.0970  3.1137    0.06
605 2025-04-24 04:05:00  3.1198  3.1199  3.1075  3.1085    0.02
606 2025-04-24 04:00:00  3.1028  3.1195  3.1028  3.1184    0.03
607 2025-04-24 03:55:00  3.1110  3.1143  3.1000  3.1034    0.03

[100 rows x 6 